In [1]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import os
import shutil
import sys

In [2]:
f = []
for (dirpath, dirnames, filenames) in os.walk('1_movies_per_genre'):
    f.extend(filenames)
    break
print (f)

['Action.csv', 'Adventure.csv', 'Animation.csv', 'Biography.csv', 'Comedy.csv', 'Crime.csv', 'Drama.csv', 'Fantasy.csv', 'History.csv', 'Horror.csv', 'Music.csv', 'Mystery.csv', 'Romance.csv', 'Sci-Fi.csv', 'Sport.csv', 'Thriller.csv', 'War.csv']


In [3]:
movie_list = pd.read_csv("1_movies_per_genre/Biography.csv")
movie_list.head()

,name,year,movie_rated,run_length,genres,release_date,rating,num_raters,num_reviews,review_url
0,Schindler's List,1993,R,3h 15min,Biography; Drama; History;,4 February 1994 (USA),8.9,1173324,1814,https://www.imdb.com/title/tt0108052/reviews/_...
1,The Wolf of Wall Street,2013,R,3h,Biography; Crime; Drama;,25 December 2013 (USA),8.2,1128148,1436,https://www.imdb.com/title/tt0993846/reviews/_...
2,Goodfellas,1990,R,2h 26min,Biography; Crime; Drama;,21 September 1990 (USA),8.7,982784,1296,https://www.imdb.com/title/tt0099685/reviews/_...
3,Braveheart,1995,R,2h 58min,Biography; Drama; History;,24 May 1995 (USA),8.3,936728,1271,https://www.imdb.com/title/tt0112573/reviews/_...
4,A Beautiful Mind,2001,PG-13,2h 15min,Biography; Drama;,4 January 2002 (USA),8.2,821709,1292,https://www.imdb.com/title/tt0268978/reviews/_...


In [4]:
csv.register_dialect('myDialect',delimiter=',', quoting=csv.QUOTE_ALL,skipinitialspace=True)
cwd = os.getcwd()
if not os.path.exists(cwd + "/2_reviews_per_movie_raw"):
    os.makedirs("2_reviews_per_movie_raw")
for filename in f:
    movie_list = pd.read_csv(f"1_movies_per_genre/{filename}")
    print (f'Scraping genre: {filename}')
    print ()
#     movie_list.head()
    for index, row in movie_list.iterrows():
        index1 = str(index + 1)
        while len(index1) < 3:
            index1 = '0' + index1
        filename = '2_reviews_per_movie_raw/' + row['name'] + ' ' + str(row['year']) + '.csv'
        if os.path.exists(cwd + f"/{filename}"):
            print (f'File {filename} already present')
            continue
        review_link = row['review_url']
        paginationKey = ""
        num_reviews = 0
        #if (index != 49): continue #uncomment line to test only 1 movie (0 - 249)
        try:
            with open(filename, "w") as f:
                writer = csv.writer(f, dialect='myDialect')
                writer.writerow(['username', 'rating', 'helpful', 'total', 'date', 'title', 'review'])
                print ('Scraping movie : ' + row['name'])
                while True:
                    page = requests.get((review_link + paginationKey), headers = {"Accept-Language": "en-US, en;q=0.5"})
                    soup = BeautifulSoup(page.content, 'html.parser')
                    review_containers = soup.find_all('div', 'review-container')
                    review_data = []
                    for review_container in review_containers:
                        rating = review_container.find('span', 'rating-other-user-rating')
                        if rating is not None:
                            rating = int(rating.span.contents[0])
                        else:
                            rating = 'Null'
                        title = review_container.find('a', 'title')
                        if title is not None:
                            title = str(title.contents[0])
                        else:
                            title = 'Null'
                        username = review_container.find('span', 'display-name-link')
                        if username is not None:
                            username = str(username.a.contents[0])
                        else:
                            username = 'Null'
                        date = review_container.find('span', 'review-date')
                        if date is not None:
                            date = str(date.contents[0])
                        else:
                            date = 'Null'
                        review = review_container.find('div', 'content')
                        if review is not None:
                            parsed_review = ''
                            for content in review.div.contents:
                                parsed_review += str(content)
                            review = parsed_review
                        else:
                            review = 'Null'
                        helpful = review_container.find('div', 'actions')
                        total = None
                        if helpful is not None:
                            wordlist = str(helpful.contents[0]).split(' out of ')
                            helpful = int(wordlist[0].strip().replace(',', ''))
                            total = int(wordlist[1].split(' ')[0].replace(',', ''))
                        else:
                            helpful = 'Null'
                            total = 'Null'
                        review_data = [username, rating, helpful, total, date, title, review]
                        writer.writerow(review_data)
                        num_reviews += 1
                    load_more_data = soup.find('div', 'load-more-data')
                    if load_more_data is not None:
                        paginationKey = str(load_more_data['data-key'])
                    else:
                        # Beautifulsoup breaks for "#&#@!" in a review in Pulp Fiction no idea why
                        # No paginationKey obtained so forcefully try from page response if available
                        paginationKey = None
                        con = str(page.content)
                        if (con.find('div class="load-more-data"') != -1):
                            paginationKey = con.split('<div class="load-more-data" data-key="')[1]
                            paginationKey = paginationKey.split('"')[0]
                            #print (paginationKey)
                        if paginationKey is None:
                            break

        except Exception as e:
            print (e)
            f.close()
            if os.path.exists(cwd + f"/{filename}"):
                os.remove(cwd + f"/{filename}")
                print (f'Removed {cwd+"/"+filename}')
            sys.exit(0)
        print ('%d / %d reviews of movie no. %s %s scraped !!!' %(num_reviews, int(row['num_reviews']), index1, row[0]))
        if ((index+1) % 10 == 0):
            print (str(index+1) + ' % movies scraped !!!')
print ('All movies scraped !!!')

Scraping genre: Action.csv

File 2_reviews_per_movie_raw/The Dark Knight 2008.csv already present
File 2_reviews_per_movie_raw/Inception 2010.csv already present
File 2_reviews_per_movie_raw/The Matrix 1999.csv already present
File 2_reviews_per_movie_raw/The Lord of the Rings: The Fellowship of the Ring 2001.csv already present
File 2_reviews_per_movie_raw/The Dark Knight Rises 2012.csv already present
File 2_reviews_per_movie_raw/Gladiator 2000.csv already present
File 2_reviews_per_movie_raw/Batman Begins 2005.csv already present
File 2_reviews_per_movie_raw/The Avengers 2012.csv already present
File 2_reviews_per_movie_raw/Star Wars: Episode IV - A New Hope 1977.csv already present
File 2_reviews_per_movie_raw/Star Wars: Episode V - The Empire Strikes Back 1980.csv already present
File 2_reviews_per_movie_raw/Avatar 2009.csv already present
File 2_reviews_per_movie_raw/Guardians of the Galaxy 2014.csv already present
File 2_reviews_per_movie_raw/V for Vendetta 2005.csv already pres